In [1]:
#Data organization dependencies
from os import remove
import pandas as pd
from google.colab import data_table
data_table.enable_dataframe_formatter()

#Web scraping dependencies
!pip install beautifulsoup4
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import urljoin
!pip install requests
import requests
!pip install justext
import justext
import re

#NLP dependencies
!pip install transformers
from transformers import pipeline
# Summarize textual content using BART in pytorch
bart_summarizer = pipeline("summarization")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [2]:
#Obtain list of links on a domain
domain = "https://www.barakatrestaurant.com"
req = Request(domain, headers={'User-Agent': 'Mozilla/5.0'})
html_page = urlopen(req)
soup = BeautifulSoup(html_page, "lxml")

links = []
for link in soup.findAll('a'):
    links.append(urljoin(domain, link.get('href')))

#Links cleanup for None types
links = [link for link in links if link is not None]  
#Links cleanup for non-URL objects
for link in links:
    if not link.startswith("http"):
        links.remove(link)
#Links cleanup for duplicate URLs
links = [link for n, link in enumerate(links) if link not in links[:n]] 
#Links cleanup for social URLs
socials = ["instagram", "facebook", "twitter", "linkedin", "tiktok", "google", "maps", "mealsy"]
clean_links = []
for link in links:
  if not any(social in link for social in socials):
    clean_links.append(link)

In [3]:
# Extract and clean text from each link 
content = []
for link in clean_links:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0'})
    paragraphs = justext.justext(response.content, justext.get_stoplist("English"))
    for_processing = []
    block_words = ["cookies", "privacy", "policy", "instagram", "facebook", "twitter", "linkedin"]
    for paragraph in paragraphs:
        paragraph = paragraph.text
        if len(paragraph) < 50:
            continue
        elif not any(block in paragraph for block in block_words):
            for_processing.append(paragraph)
    for_processing = ' '.join(for_processing[:5])
    content.append(for_processing)

df = pd.DataFrame({'link': clean_links, 'content': content})

In [4]:
# Summarize content for meta descriptions
summarized_descriptions = []
for item in content:
    print(item)
    print(bart_summarizer(item, min_length = 20, max_length = 50))
    summarized_descriptions.append(bart_summarizer(item, min_length = 20, max_length = 50))


Whether you are a Lebanese food enthusiast or want to try it for the first time, we welcome you! We love providing the genuine taste of authentic Lebanese food for our guests and to bring them new and exciting dining experiences. Browse through our fresh and tasty menu. You will find a large variety of dishes made to satisfy any taste! Simply order online on our website. No matter where you are, our food can become available to you! 1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia, Ontario -
(519) 850-8080
[{'summary_text': ' We love providing the genuine taste of authentic Lebanese food for our guests . You will find a large variety of dishes made to satisfy any taste! Simply order online on our website .'}]
Whether you are a Lebanese food enthusiast or want to try it for the first time, we welcome you! We love providing the genuine taste of authentic Lebanese food for our guests and to bring them new and exciting dining experiences. Browse through o

Your max_length is set to 50, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


2020 © Barakat Restaurant. Designed and developed by
WWW.MEALSY.CA Privacy Policy Loyalty Program


Your max_length is set to 50, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


[{'summary_text': ' 2020 © Barakat Restaurant. 2020 . 2020 © Restaurant.com . Privacy Policy Loyalty Program .'}]
1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia, Ontario -
(519) 850-8080 1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia, Ontario -
(519) 850-8080 2020 © Barakat Restaurant. Designed and developed by
WWW.MEALSY.CA Privacy Policy Loyalty Program
[{'summary_text': ' 1149 Western Rd London, ON N6G 5K8 - \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia .'}]
Roman Lettuce, Tomato, Parsley, Cucumber, Garlic, Onion, Lemon Juice, Olive Oil, Toasted Pita Fattoush Salad with Chicken Shawarma on the Top and Your Choice of Garlic Sauce or Cabbage Sauce or Cabbage 1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia, Ontario -
(519) 850-8080 1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080
[{'summary_text': ' Roman Lettuce, Tomato, Parsley, Cucu

Your max_length is set to 50, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


2020 © Barakat Restaurant. Designed and developed by
WWW.MEALSY.CA Privacy Policy Loyalty Program


Your max_length is set to 50, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


[{'summary_text': ' 2020 © Barakat Restaurant. 2020 . 2020 © Restaurant.com . Privacy Policy Loyalty Program .'}]
Whether you are a Lebanese food enthusiast or want to try it for the first time, we welcome you! We love providing the genuine taste of authentic Lebanese food for our guests and to bring them new and exciting dining experiences. Browse through our fresh and tasty menu. You will find a large variety of dishes made to satisfy any taste! Simply order online on our website. No matter where you are, our food can become available to you! 1149 Western Rd London, ON N6G 5K8 -
(519) 850-8080 1362 Lambton Mall Rd #11 Sarnia, Ontario -
(519) 850-8080
[{'summary_text': ' We love providing the genuine taste of authentic Lebanese food for our guests . You will find a large variety of dishes made to satisfy any taste! Simply order online on our website .'}]
We may collect personal identification information from users in a variety of ways, including, but not limited to, when users visit 

In [273]:
from pandas.core.internals.managers import T
from numpy.ma.core import count
#Retrieve values from list of dictionaries in summarized_descriptions
meta_descriptions = [summary[0]["summary_text"] for summary in summarized_descriptions]

# General data cleaning
meta_descriptions_clean1 = []
for description in meta_descriptions:
  #Clean up leading and trailing spaces:
  description = description.strip()
  #Clean up excessive spaces
  description = re.sub(' +', ' ', description)
  #Clean up punctuation spaces
  description = description.replace(' .', '.')
  #Clean up incomplete sentences
  if "." in description and not description.endswith("."):
    description = description.split(".")[:-1]
    description = description[0]
  meta_descriptions_clean1.append(description)

meta_descriptions_clean2 = []
#Add a period to all sentences (if missing)
for description in meta_descriptions_clean1:
  if not description.endswith("."):
    description = description + "."
  meta_descriptions_clean2.append(description)

#Adhere to meta description length of approx. 160 characters
#Find the index of the punctuation character desired
def find_all(string, character):
  index = string.find(character)
  while index != -1:
    yield index
    index = string.find(character, index + 1)

#Store truncation points
truncation_points = []
character = "."
for description in meta_descriptions_clean2:
  indexes = list(find_all(description, character))
  truncation_points.append(indexes)

# Auto truncate
meta_descriptions_clean3 = []
character = "."
for description in meta_descriptions_clean2:
  if len(description) > 160 and description.count(character) > 1:
    split = description.split(character)[:-2]
    description = character.join(split)
    if not description.endswith("."):
      description = description + "."
    meta_descriptions_clean3.append(description)
  else:
    meta_descriptions_clean3.append(description)
 
#Verify length adherence for non-truncated descriptions
len_meta_descriptions = []
for description in meta_descriptions_clean2:
  len_meta_descriptions.append(len(description))

#Verify length adherence for auto-truncated descriptions
truncated_len_meta_descriptions = []
for description in meta_descriptions_clean3:
  truncated_len_meta_descriptions.append(len(description))

# Organize results into dataframe    
df = pd.DataFrame({'link': clean_links, 'content': content, 'meta_descriptions': meta_descriptions_clean2, 'description_length': len_meta_descriptions, 'truncation_points': truncation_points, 'truncated_descriptions': meta_descriptions_clean3, 'truncated_length': truncated_len_meta_descriptions})
df

,link,content,meta_descriptions,description_length,truncation_points,truncated_descriptions,truncated_length
0,https://www.barakatrestaurant.com/index.html,Whether you are a Lebanese food enthusiast or ...,We love providing the genuine taste of authent...,181,"[77, 180]",We love providing the genuine taste of authent...,78
1,https://www.barakatrestaurant.com#!,Whether you are a Lebanese food enthusiast or ...,We love providing the genuine taste of authent...,181,"[77, 180]",We love providing the genuine taste of authent...,78
2,https://www.barakatrestaurant.com/story.html,Barakat is a real Lebanese style restaurant lo...,Barakat is a real Lebanese style restaurant lo...,167,"[84, 166]",Barakat is a real Lebanese style restaurant lo...,85
3,https://www.barakatrestaurant.com/NUTRJITION.html,"Lebanese cuisine includes the vegetarian, vega...","Lebanese cuisine includes the vegetarian, vega...",217,"[62, 105, 170, 216]","Lebanese cuisine includes the vegetarian, vega...",171
4,https://www.barakatrestaurant.com/catering.html,Let us take the cooking off your hands! We spe...,We specialize in catering unique menus special...,126,"[71, 125]",We specialize in catering unique menus special...,126
5,https://www.barakatrestaurant.com/contact.html,"1149 Western Rd London, ON N6G 5K8 Tel: (519) ...","1149 Western Rd London, ON N6G 5K8 Tel: (519) ...",96,[95],"1149 Western Rd London, ON N6G 5K8 Tel: (519) ...",96
6,https://www.barakatrestaurant.com/APPETIZERS.html,Fries and Chicken Shawarma with Hummus and Cab...,Fries and Chicken Shawarma with Hummus and Cab...,133,"[80, 132]",Fries and Chicken Shawarma with Hummus and Cab...,133
7,https://www.barakatrestaurant.com/MEALS.html,A Combination of Beef or Chicken Shawarma with...,"Two Sticks of Shish Kabob, Tawook and Kafta, S...",101,[100],"Two Sticks of Shish Kabob, Tawook and Kafta, S...",101
8,https://www.barakatrestaurant.com/PLATES.html,A Combination of Shish Tawook and Shish Kabob ...,A Combination of Shish Tawook and Shish Kabob ...,127,[126],A Combination of Shish Tawook and Shish Kabob ...,127
9,https://www.barakatrestaurant.com/WRAPS.html,2020 © Barakat Restaurant. Designed and develo...,2020 © Barakat Restaurant. 2020. 2020 © Restau...,87,"[25, 31, 50, 54, 86]",2020 © Barakat Restaurant. 2020. 2020 © Restau...,87
